In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random
import re

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
twt_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/train_featured.csv')
twt_data.head()

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,NaN,1,0,0,0,deeds reason earthquake may allah forgive us,68,4.31,25,7,6,13
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,NaN,NaN,0,0,0,0,forest fire near la ronge sask canada,37,4.43,13,1,0,7
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,NaN,NaN,0,0,0,0,residents asked shelter place notified officer...,130,4.95,45,9,11,22
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,NaN,1,0,5,0,people receive wildfires evacuation orders cal...,56,7.14,24,1,1,7
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,NaN,2,0,0,0,got sent photo ruby alaska smoke wildfires pou...,85,4.38,25,3,7,16


In [3]:
pattern = re.compile("(?P<url>https?://[^\s]+)")
def remove_link(twt):
    return pattern.sub("r ", twt)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text  import TfidfVectorizer

In [5]:
train, test = train_test_split(twt_data, test_size=0.2)

In [6]:
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(twt_data['clean_text'])
X_train_counts.shape

(7613, 14190)

In [7]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(7613, 14190)

In [8]:
count_vect.get_feature_names()

['aa',
 'aaa',
 'aaaand',
 'aaalll',
 'aaarrrgghhh',
 'aaemiddleaged',
 'aal',
 'aan',
 'aannnd',
 'aar',
 'aashiqui',
 'ab',
 'aba',
 'abandon',
 'abandoned',
 'abandoning',
 'abbandoned',
 'abbott',
 'abbruchsimulator',
 'abbswinston',
 'abc',
 'abceyewitness',
 'abcnews',
 'abcs',
 'abe',
 'aberdeen',
 'aberystwythshrewsbury',
 'abes',
 'abha',
 'abia',
 'ability',
 'abject',
 'ablaze',
 'able',
 'ableg',
 'aboard',
 'abomb',
 'abombed',
 'abomination',
 'abortion',
 'abortions',
 'abouts',
 'abq',
 'abs',
 'absence',
 'absolute',
 'absolutely',
 'abstorm',
 'abstract',
 'absurd',
 'absurdly',
 'abuse',
 'abused',
 'abuses',
 'abusing',
 'ac',
 'academia',
 'acc',
 'accept',
 'accepte',
 'accepts',
 'access',
 'accident',
 'accidentally',
 'accidentalprophecy',
 'accidently',
 'accidents',
 'accompanying',
 'according',
 'accordingly',
 'account',
 'accountable',
 'accounts',
 'accuracy',
 'accused',
 'accuses',
 'accustomed',
 'acdelco',
 'ace',
 'acenewsdesk',
 'acesse',
 'achedin

In [9]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

In [10]:
text_clf = text_clf.fit(train.clean_text, train.target_label)

In [11]:
predicted = text_clf.predict(test.clean_text)
np.mean(predicted == test.target_label)

0.8161523309258043

In [12]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, random_state=42)),
                        ])

In [13]:
text_clf_svm = text_clf_svm.fit(train.clean_text, train.target_label)

In [14]:
predicted_svm = text_clf_svm.predict(test.clean_text)
np.mean(predicted_svm == test.target_label)

0.7701904136572554

In [15]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
             }
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(train.clean_text, train.target_label)

In [16]:
gs_clf.best_score_

0.7788177339901478

In [17]:
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

In [18]:
text_clf_improved = Pipeline([('vect', CountVectorizer()),
                              ('clf', MultinomialNB()),
                             ])

In [19]:
text_clf = text_clf_improved.fit(train.clean_text, train.target_label)

In [20]:
predicted = text_clf_improved.predict(test.clean_text)
np.mean(predicted == test.target_label)

0.8049901510177282

In [21]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train.clean_text)
tf_idf_train = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())
tf_idf_train

,aa,aaaand,aaalll,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,...,zone,zones,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6086,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6087,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6088,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [22]:
test_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/test_featured.csv')
test_data.head()

,id_original,keyword_original,location_original,text_original,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count
0,0,NaN,NaN,just happened terrible car crash,0,NaN,NaN,0,0,0,0,happened terrible car crash,34,4.83,10,2,2,6
1,2,NaN,NaN,heard about earthquake is different cities sta...,3,#earthquake,NaN,1,0,0,0,heard earthquake different cities stay safe ev...,61,5.89,24,1,2,9
2,3,NaN,NaN,there is forest fire at spot pond geese are f...,2,NaN,NaN,0,0,0,0,forest fire spot pond geese fleeing across str...,94,4.00,31,7,9,19
3,9,NaN,NaN,apocalypse lighting spokane wildfires,3,#spokane #wildfires,NaN,2,0,0,0,apocalypse lighting spokane wildfires,37,8.50,12,0,0,4
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan,0,NaN,NaN,0,0,2,0,typhoon soudelor kills china taiwan,42,5.14,14,2,2,7


In [23]:
text_clf = text_clf.fit(twt_data.clean_text, twt_data.target_label)

In [24]:
predicted = text_clf.predict(test_data.clean_text)
predicted

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [25]:
test_data[test_data.clean_text != test_data.clean_text]

,id_original,keyword_original,location_original,text_original,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count
13,43,NaN,NaN,what if,2,NaN,NaN,0,0,0,0,NaN,7,3.00,2,1,2,2


In [ ]:
test_data['clean_text'].fillna(" ", inplace=True)

In [ ]:
test_data[test_data.clean_text != test_data.clean_text]

In [ ]:
test_data['target'] = predicted
test_data[['id_original', 'target']].rename(columns={'id_original': 'id'}).to_csv('~/Documents/Datos/DataSets/TP2/res_NB_1.csv', index=False)